<h1 align = center> Limpieza de la Base de Datos </h1>

Usualmente las bases de datos presentan muchos posibles problemas, como datos faltantes y datos atipicos que se deben tratar para poder utilizar la base de datos.

## Tratamiento de Na's

Lo primero que se hace es realizar la lectura de la base de datos

In [ ]:
url1   <- "https://raw.githubusercontent.com/joanby/r-course/master/data/tema1/missing-data.csv"
data1R <- read.csv(url1, row.names=NULL)
head(data1R, 5)

### Estandarizar los NAN

In [ ]:
data1R$Income[data1R$Income == 0]          <- NA
data1R$Phone_type[data1R$Phone_type == ""] <- NA
data1R$Car_type[data1R$Car_type == ""]     <- NA

head(data1R, 5)

### Observación de NA's e Identificación de Patrones
Contar el número de NAN por variable (columna).

In [ ]:
colSums(is.na(data1R))

library(dplyr)
summarise_all(data1R, funs(sum(is.na(.))))

Dependiendo del contexto en ocasiones el ruido que introduce una variable cuando contiene muchos valores nulos es demasiado, por lo tanto es una buena tecnica observar el patron que siguen lis NA's.

In [ ]:
url3         <- "https://raw.githubusercontent.com/joanby/r-course/master/data/tema1/housing-with-missing-value.csv"
data3R       <- read.csv(url3)
housing.data <- data3R

In [ ]:
library(mice)
md.pattern(housing.data)

In [ ]:

library(VIM)

aggr(housing.data,
     col= c('blue', 'red'),
     numbers = TRUE, 
     sortVars = TRUE,
     labels = names(housing.data),
     cex.axis = 0.75,
     gap = 1,
     ylab = c("Histograma de NAs", "Patrón")
)     

### Eliminar los NA

Es posible que eliminar los Na, la forma más simple de hacerlo es eliminar todas las filas que contengan algún valor nulo.

In [ ]:
ingresos <- na.omit(data1R)
head(ingresos, 5)

In [ ]:
ingresos <- data1R[complete.cases(data1R),]
head(ingresos, 5)

Aunque tambien esta la opción de eliminar las filas que tengan un valor nulo en una variable concreta.

In [ ]:
ingresos <- data1R[!is.na(data1R$Income),]
colSums(is.na(ingresos))
head(ingresos, 5)

Támbien existe la posibilidad de que alguna variable tenga tantos NA's que sea más conveniente eliminarla.

In [ ]:
ingresos$Phone_type <- NULL
head(ingresos, 5)

### Imputación de NA's

#### Imputación por media

In [ ]:
mean(data1R$Income)
mean(data1R$Income, na.rm=T)

sd(data1R$Income)
sd(data1R$Income, na.rm=T)

In [ ]:
ingresos             <- data1R
ingresos$Income.mean <- ifelse(is.na(ingresos$Income), 
                           mean(ingresos$Income, na.rm=TRUE),
                           ingresos$Income
                           )
head(ingresos,5)

In [ ]:
mean(ingresos$Income.mean)
sd(ingresos$Income.mean)

Tambien existe un paquete llamado *Hmisc* que permite relizar esto sin necesidad e crear una función, además permite realizar un cambio en el criterio de imputación

In [ ]:
housing.data <- data3R
summary(housing.data$ptratio)

In [ ]:
library(Hmisc)
housing.data$ptratio <- impute(housing.data$ptratio, mean)
summary(housing.data$ptratio)

Además tambien es posible realizar la imputación 

In [ ]:
housing.data$ptratio <- impute(housing.data$ptratio, median)
summary(housing.data$ptratio)

In [ ]:
housing.data <- data3R
housing.data$ptratio <- impute(housing.data$ptratio, 18)
summary(housing.data$ptratio)

#### Computar Valores aleatorios

Cuanto tengo valores faltantes, tanto en variables númericas como categoricas de forma que por medio de un muestreo aleatorio se reeemplazan los valores faltantes. Para esto es conveniente construir una función

In [ ]:
random.impute        <- function(x) {
    missing          <- is.na(x)
    n.missing        <- sum(missing)
    x.obs            <- x[!missing]
    imputed          <- x
    imputed[missing] <- sample(x.obs, n.missing, replace=TRUE)
    return(imputed)
}

Primero uye se construye un vector con los valores nulos **missing**, y los sumamos para saber cuales valores hay que reemplazar **n.missing**, luego  se saca un vector de los valores no nulos **x.obs**, **imputed** devuelve por defecto los valores que habian en los valores no nulos y finalmente **imputed[missing]** Permite reemplazar aletoriamente los valores nulos con valores observados.

In [ ]:
ingresos                    <- data1R
ingresos$Phone_type.imputed <- random.impute(ingresos$Phone_type)
colSums(is.na(ingresos))

Cuando los datos que se tienen en la base de datos fueron tomados de forma totalmente aleatoria, es posible optar por realizar una imputación aleatoria de todos los datos de la variable

In [ ]:
random.impute.data.frame <- function(dataframe, cols){
  names                  <- names(dataframe)
  for(col in cols){
    name                 <- paste(names[col], "imputed", sep = ".")
    dataframe[name]      = random.impute(dataframe[,col])
  }
  dataframe
}

In [ ]:
ingresos <- data1R
ingresos <- random.impute.data.frame(ingresos, c(1,2))
colSums(is.na(ingresos))

In [ ]:
drop_na        <- c("rad")
housing.data.2 <- housing.data[ 
  complete.cases(housing.data[,!(names(housing.data))%in% drop_na]),]
head(housing.data.2, 5)
dim(housing.data.2)

#### Imputación con Modelos Predictivos

En ocasiones cuando un reemplazo de valores aleatorios no es el adecuado, y el uso de la media y mediana genera mucho ruido es usual utilizar modelos predictivos para generar los valores.

In [ ]:
library(mice)
housing.data <- data3R
head(housing.data, 5)
dim(housing.data)

In [ ]:
colSums(is.na(housing.data))

Para hacer esto se hace uso del paquete **mice**

* m: Es el número de imputaciones que quiero hacer
* maxit: Es el número maximo de iteraciones
* seed: Es la semilla; todos los métodos de reemplazo del paquete son aleatorios, y la semilla va cambiando si esta no se fija cambiando los valores con los que se imputo.
* method: Metodo de reemplazo
    + pmm - comparación predictiva de medias
    + logreg - regresión logistica
    + polyreg - regresión logística politómica
    + polr - modelo de probabilidades proporcionales
    
**nota:** Para generar números pseudo aleatorios se utiliza un generador lineal congruencial

In [ ]:
columns <- c("ptratio", "rad")

##### Imputación por Comparación Predictiva de medias

In [ ]:
imputed_data <- mice(housing.data[,names(housing.data) %in% columns],
                     m = 5, 
                     maxit = 50, 
                     method = "pmm",
                     seed = 2018)

In [ ]:
summary(imputed_data)

Despues de generar los valores de forma aleatoria se reemplazan los NA's por los valores generados

In [ ]:
housing.data$ptratio <- mice::complete(imputed_data)$ptratio
housing.data$rad <- mice::complete(imputed_data)$rad
colSums(is.na(housing.data))
anyNA(housing.data)

#### Imputación automatica

Es posible utilizar el paquete **hmisc** para que este elija de forma automatica elmétodo e impute los NA's.

In [ ]:
impute_arg <- aregImpute(~ptratio + rad, data = housing.data, n.impute = 5)

impute_arg

impute_arg$imputed$rad

## Valores Duplicados

En ocasiones encontramos valores duplicados, o registros duplicados en una valiable, lo primero es verificar si hay registros duplicados en el data frame.

In [ ]:
family.salary <- c(40000, 60000, 50000, 80000, 60000, 70000, 60000)
family.size   <- c(4, 3, 2, 2, 3, 4, 3)
family.car    <- c("Lujo", "Compacto", "Utilitario", "Lujo", 
               "Compacto", "Compacto", "Compacto")
family        <- data.frame(family.salary, family.size, family.car)
family

In [ ]:
duplicated(family)
family[duplicated(family),]

Ahora cuando se encuentran valores duplicados en un data frame, estos se pueden eliminar utilizando la función unique.

In [ ]:
family.unique <- unique(family)
family.unique

## Tratamiento de Outliers

En estadistica se conoce como *rango intercuartilico* $RI$ a la distancia que hay entre el cuartil uno $Q_1$ y el cuartil tres $Q_3$, y se considera un outlier un dato que este por fuera de $[Q_1-1.5RI, Q_3+1.5RI]$.

### Identificación de Outliers

Para identificar si hay valores atipicos en un grupo de datos una de las herramientas más utilies es el uso de boxplots.

In [ ]:
url5       <- "https://raw.githubusercontent.com/joanby/r-course/master/data/tema1/ozone.csv"
data5R     <- read.csv(url5, stringsAsFactors=F)
ozone.data <- data5R
head(ozone.data, 5)

In [ ]:
boxplot(ozone.data$pressure_height, 
        main = "Pressure Height",
        boxwex = 1.5)$out

El parametro out se utiliza para que aparezca cuales son las observaciones que son outliers en la variable analizada.

En ocasiones es importante observar si los outlaiers tienen realción con otra variable.

In [ ]:
boxplot(ozone_reading ~Month,
        data = ozone.data,
        main = "Ozone reading per Month")$out

mtext("Así se puede poner una descripción")

Exiten tambien otros métodos, para identicar los outlaiers

### Imputación de Atípicos

In [ ]:
replace_outliers <- function(x, removeNA=TRUE){
    q1   <- quantile(x,probs=0.25, na.rm=removeNA)
    q3   <- quantile(x,probs=0.75, na.rm=removeNA)
    ri   <- q3 - q1   
    caps <- quantile(x, probs = c(.05, .95), na.rm = removeNA)  
    x[x<q1-1.5 * ri] <- caps[1]
    x[x>q3+1.5 * ri] <- caps[2]
    x
    }

In [ ]:
capped_pressure_height <- replace_outliers(ozone.data$pressure_height)
head(capped_pressure_height,5)

In [ ]:
par(mfrow = c(1,2))
boxplot(ozone.data$pressure_height, main = "Presión con outliers")
boxplot(capped_pressure_height, main = "Presión sin outliers")

### Guardar la nueva base de datos

Por motivos de replicabilidad es adecuado generar una nueva base de datos en lugar de modificar la original, aal momento de continuar con el analisis.

In [ ]:
write.csv(MyData, file = "MyData.csv")